<a href="https://colab.research.google.com/github/gerasimos-matidis/my_icdar/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO:
- Load the model
- Get the input size for the model inputs
- Divide the input image into the appropriate number of patches, with patch size equal to the input size of the model
- Pass all the patches through the network to make predictions
- Merge all the outputs of the patches so that they create an predicted image with the size of the input image  
- Visualize the results in a "Input image - Predicted mask - Ground truth" plot.